In [2]:
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

import torch as t
from torch.autograd import Variable as V
import torch.nn as nn
import torch.optim as optim

t.manual_seed(777)      # reproducibility

#### 1. input data

In [7]:
idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]    # hihell
y_data = [1, 0, 2, 3, 3, 4]      # ihello

# As we have one batch of samples, we will change them to variables only once
inputs = V(t.LongTensor(x_data))
labels = V(t.LongTensor(y_data))

if t.cuda.is_available():
    inputs = inputs.cuda()
    labels = labels.cuda()

print("inputs size: ", inputs.size())
print("inputs labels: ", labels.size())

inputs size:  torch.Size([1, 6])
inputs labels:  torch.Size([6])


#### 2. define model

In [4]:
# parameters setting
num_classes = 5
input_size = 5
embedding_size = 10 # embedding size
hidden_size = 5     # output from the RNN(5) to directly predict one-hot
batch_size = 1      # one sentence
sequence_length = 6 # |ihello| = 6
num_layers = 1      # one-layer rnn

In [8]:
# net model
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(input_size=embedding_size,
                          hidden_size=5,
                          batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Initialize hidden and cell states
        # (batch, num_layers * num_direction, hidden_size) for batch_first=True
        h_0 = V(t.zeros(x.size(0), num_layers, hidden_size))
        if t.cuda.is_available():
            h_0 = h_0.cuda()
        
        emb = self.embedding(x)
        emb = emb.view(batch_size, sequence_length, -1)
        
        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # h_0: (batch, num_layers * num_directions, hidden_size)
        out, _ = self.rnn(emb, h_0)
        return self.fc(out.view(-1, num_classes))        

#### 3. train model

In [11]:
# Instantiate RNN model
model = Model()
if t.cuda.is_available():
    model.cuda()
print(model)

# Set loss and optimizer function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

# Train the model
for epoch in range(10):
    outputs = model(inputs)
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    
    loss.backward()
    optimizer.step()
   
    _, idx = outputs.max(1)
    
    predicted = ''.join([idx2char[eachidx] for eachidx in idx.cpu().data.numpy()])
    print("Predicted string: ", predicted)
    
    print("Epoch: %d, Loss: %1.3f" % (epoch + 1, loss.data[0]))

print("Learning finished!!!")
    

Model(
  (embedding): Embedding(5, 10)
  (rnn): RNN(10, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=5)
)
Predicted string:  ihiihh
Epoch: 1, Loss: 1.625
Predicted string:  ehello
Epoch: 2, Loss: 1.170
Predicted string:  ehelll
Epoch: 3, Loss: 0.937
Predicted string:  ehello
Epoch: 4, Loss: 0.751
Predicted string:  ehello
Epoch: 5, Loss: 0.608
Predicted string:  ehello
Epoch: 6, Loss: 0.503
Predicted string:  ehello
Epoch: 7, Loss: 0.393
Predicted string:  ihello
Epoch: 8, Loss: 0.306
Predicted string:  ihello
Epoch: 9, Loss: 0.238
Predicted string:  ihello
Epoch: 10, Loss: 0.184
Learning finished!!!
